In [4]:
!pip install kaggle

Importing the Dependences

In [12]:
import os

In [13]:
import json

In [14]:
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**Data Collection **

In [15]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [16]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  sample_data


In [17]:
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [18]:
data=pd.read_csv("IMDB Dataset.csv")

In [19]:
data.shape


(50000, 2)

In [20]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [21]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [22]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [23]:
data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)

<ipython-input-23-329ec005e21e>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)


In [24]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [25]:
data["sentiment"].value_counts()


,count
sentiment,
1,25000
0,25000


In [26]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=42)

In [27]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [28]:
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train=pad_sequences(tokenizer.texts_to_sequences(train_data["review"]),maxlen=200)
X_test=pad_sequences(tokenizer.texts_to_sequences(test_data["review"]),maxlen=200)

In [29]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [30]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [31]:
Y_train=train_data["sentiment"]
Y_test=test_data["sentiment"]

In [32]:
print(Y_train.shape)
print(Y_test.shape)

(40000,)
(10000,)


*Long short term memory*

In [33]:
model=Sequential()
model.add(Embedding(5000,input_length=200,output_dim=128))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation="sigmoid"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [34]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [35]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [36]:
#training the model
model.fit(X_train,Y_train,epochs=5,batch_size=64,validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 131s 244ms/step - accuracy: 0.7005 - loss: 0.5500 - val_accuracy: 0.8169 - val_loss: 0.4143
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 135s 243ms/step - accuracy: 0.8435 - loss: 0.3732 - val_accuracy: 0.8497 - val_loss: 0.3577
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 146s 251ms/step - accuracy: 0.8778 - loss: 0.3072 - val_accuracy: 0.8313 - val_loss: 0.3878
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 147s 261ms/step - accuracy: 0.8735 - loss: 0.3109 - val_accuracy: 0.8652 - val_loss: 0.3352
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 136s 249ms/step - accuracy: 0.9024 - loss: 0.2485 - val_accuracy: 0.8674 - val_loss: 0.3404


In [37]:
loss,accuracy=model.evaluate(X_test,Y_test)
print(f"test losss:{loss}")
print(f"test accuracy:{accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 84ms/step - accuracy: 0.8671 - loss: 0.3376


In [39]:
def predict_sentiment(review):
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment="positive" if prediction>0.5 else "negative"
  return sentiment

In [40]:
new_review="This movie was fantastic. I loved it "
sentiment=predict_sentiment(new_review)
print(f"The sentiment of the review is {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
The sentiment of the review is positive


In [42]:
new_review="This movie was not that good "
sentiment=predict_sentiment(new_review)
print(f"The sentiment of the review is {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
The sentiment of the review is negative
